In [1]:
m = 5
s = 6

import torch

from itertools import product

sub_lists = [range(m) for _ in range(s)]

new_tuples = list(product(*sub_lists))  # s=3 ->  [1], [1,2], [1,2,3]
print(sub_lists)
print(len(new_tuples))

print(new_tuples[:2])
new_tuples = list(sum(new_tuples, ()))

new_tuples[:2*s]

new_tuples = torch.tensor(new_tuples)

new_tuples = new_tuples.reshape(-1, m, s) 

# 10 = 2*(4 + 1) 5 patch, 1 info, 4 non-info a,b 00 00 00 00 a0 

new_tuples[0]

[range(0, 5), range(0, 5), range(0, 5), range(0, 5), range(0, 5), range(0, 5)]
15625
[(0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1)]


tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 2],
        [0, 0, 0, 0, 0, 3],
        [0, 0, 0, 0, 0, 4]])

In [3]:
from src.SRHM.reference_code.hierarchical import *
from src.SRHM.srhm import *

num_layers=3
num_classes=3
num_features=5
seed=0
max_dataset_size=10
s=2
s0=3

trainset = SparseRandomHierarchyModel(
    num_features=num_features,
    m=1,  # features multiplicity
    num_layers=num_layers,
    num_classes=num_classes,
    s=s, # tuples size
    s0=s0,
    input_format="long",
    whitening=0,
    seed=seed,
    train=True,
    transform=None,
    testsize=-1,
    max_dataset_size=max_dataset_size
)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=1, shuffle=True, num_workers=0)

# batch, _, _ 

for elem in list(trainloader)[:10]:
    print("at tensor level")
    for subelem in elem:
        print("feature shape:", elem[0].shape)
        print("unique features:", torch.unique(elem[0][0], dim=0))
        print("feature:", elem[0])
        print("label:", elem[1])
        print()

trainset = RandomHierarchyModel(
    num_features=num_features,
    m=1,  # features multiplicity
    num_layers=num_layers,
    num_classes=num_classes,
    s=s, # tuples size
    input_format="long",
    whitening=0,
    seed=seed,
    train=True,
    transform=None,
    testsize=-1,
    max_dataset_size=max_dataset_size
)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=1, shuffle=True, num_workers=0)

# batch, _, _ 

for elem in list(trainloader)[:10]:
    print("at tensor level")
    for subelem in elem:
        print("feature shape:", elem[0].shape)
        print("unique features:", torch.unique(elem[0][0], dim=0))
        print("feature:", elem[0])
        print("label:", elem[1])
        print()



SRHM: Using sparsity type B
at tensor level
feature shape: torch.Size([1, 24])
unique features: tensor([0, 1, 2, 3])
feature: tensor([[3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0]])
label: tensor([2])

feature shape: torch.Size([1, 24])
unique features: tensor([0, 1, 2, 3])
feature: tensor([[3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0]])
label: tensor([2])

at tensor level
feature shape: torch.Size([1, 24])
unique features: tensor([0, 1, 2, 3, 4, 5])
feature: tensor([[0, 2, 0, 0, 3, 0, 0, 0, 5, 0, 4, 0, 3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0]])
label: tensor([1])

feature shape: torch.Size([1, 24])
unique features: tensor([0, 1, 2, 3, 4, 5])
feature: tensor([[0, 2, 0, 0, 3, 0, 0, 0, 5, 0, 4, 0, 3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0]])
label: tensor([1])

at tensor level
feature shape: torch.Size([1, 24])
unique features: tensor([0, 1, 2, 3])
feature: tensor([[0, 2, 0, 0, 3, 0, 0, 2, 0, 0, 3, 0, 3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0]])
label: 

In [4]:
from hierarchy_learning.datasets.hierarchical import sample_hierarchical_rules, sample_data_from_paths

num_layers=3
num_classes=2
num_features=3
seed=0
max_dataset_size=20
s=4

Pmax = m ** ((s ** num_layers - 1) // (s - 1)) * num_classes

paths, _ = sample_hierarchical_rules(
    num_features, num_layers, m, num_classes, s, seed=seed
)

max_dataset_size_fix = max_dataset_size
if max_dataset_size_fix is None or max_dataset_size_fix > Pmax:
    max_dataset_size_fix = Pmax

g = torch.Generator()
g.manual_seed(0)

if Pmax < 5e6:  # there is a crossover in computational time of the two sampling methods around this value of Pmax
    samples_indices = torch.randperm(Pmax, generator=g)[:max_dataset_size]
else:
    samples_indices = torch.randint(Pmax, (2 * max_dataset_size,), generator=g)
    samples_indices = torch.unique(samples_indices)
    perm = torch.randperm(len(samples_indices), generator=g)[:max_dataset_size]
    samples_indices = samples_indices[perm]


x, targets = sample_data_from_paths(
    samples_indices, paths, m, num_classes, num_layers, s, seed=seed, seed_reset_layer=43
)

ModuleNotFoundError: No module named 'hierarchy_learning'

In [60]:
for elem in paths:
    print(elem)

tensor([0, 1])
tensor([[[0, 2, 2, 0],
         [1, 2, 1, 0],
         [0, 2, 2, 2],
         [2, 2, 1, 0],
         [0, 1, 0, 0]],

        [[2, 2, 1, 1],
         [1, 0, 0, 2],
         [1, 1, 2, 2],
         [1, 1, 0, 1],
         [2, 0, 2, 0]]])
tensor([[[2, 1, 0, 2],
         [2, 1, 1, 2],
         [2, 0, 1, 1],
         [1, 1, 0, 2],
         [1, 2, 0, 1]],

        [[1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 2, 2],
         [2, 2, 1, 2],
         [0, 2, 0, 0]],

        [[1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 2, 2],
         [2, 2, 1, 2],
         [0, 2, 0, 0]],

        [[2, 1, 0, 2],
         [2, 1, 1, 2],
         [2, 0, 1, 1],
         [1, 1, 0, 2],
         [1, 2, 0, 1]],

        [[1, 2, 2, 0],
         [2, 1, 1, 0],
         [2, 2, 0, 0],
         [0, 0, 2, 0],
         [0, 1, 1, 2]],

        [[1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 2, 2],
         [2, 2, 1, 2],
         [0, 2, 0, 0]],

        [[1, 2, 2, 0],
         [2, 1, 1, 0],
   

In [73]:
print("Number of features from params :", num_features)
print("Number of features from data   :", torch.unique(x).numel())
print("---")
print("Number of classes from params: ", num_classes)
print("Number of classes from data  : ", torch.unique(targets).numel())
print("---")

for xx, y in zip(x, targets):
    print(xx.tolist(), y.item())

Number of features from params : 3
Number of features from data   : 3
---
Number of classes from params:  2
Number of classes from data  :  2
---
[0, 2, 0, 2, 0, 0, 1, 2, 1, 1, 2, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 2, 0, 1, 2, 1, 0, 2, 0, 2, 0, 1, 1, 2, 0, 1, 2, 0, 2, 0, 0, 1, 2, 0, 2, 0, 0, 1, 1, 1, 0] 0
[2, 2, 1, 0, 1, 1, 1, 0, 0, 2, 2, 0, 2, 0, 2, 0, 1, 0, 2, 2, 2, 0, 2, 0, 1, 2, 0, 2, 0, 2, 0, 2, 1, 1, 1, 2, 2, 0, 2, 0, 1, 2, 0, 2, 1, 1, 0, 1, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2] 1
[2, 1, 0, 2, 0, 0, 1, 2, 2, 1, 0, 2, 1, 2, 1, 0, 1, 1, 2, 0, 1, 0, 2, 2, 1, 1, 2, 0, 2, 1, 0, 2, 2, 0, 2, 0, 2, 1, 0, 2, 1, 0, 2, 2, 1, 2, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 1, 2, 1, 0, 1, 0, 2, 2] 0
[0, 0, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 1, 1, 2, 2, 2, 1, 0, 1, 2, 0, 2, 0, 2, 0, 2, 0, 0, 1, 2, 1, 2, 1, 0, 1, 2, 0, 2, 0, 2, 2, 0, 0, 0, 1, 2] 1
[1, 0, 2, 2, 2, 2, 1, 0, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 1, 0, 0, 2, 0, 2, 1

In [72]:
# pairs

def pairs_to_num(xi, n):

    """
        Convert one long input with n-features encoding to n^2 pairs encoding.
    """
    ln = len(xi)
    xin = torch.zeros(ln // 2)
    for ii, xii in enumerate(xi):
        xin[ii // 2] += xii * n ** (1 - ii % 2)
    return xin

def pairing_features(x, n):
    """
        Batch of inputs from n to n^2 encoding.
    """
    xn = torch.zeros(x.shape[0], x.shape[-1] // 2)
    for i, xi in enumerate(x.squeeze()):
        xn[i] = pairs_to_num(xi, n)
    return xn

x_pairs = pairing_features(x, num_features)
print(x_pairs[:10])

tensor([[2., 2., 0., 5., 4., 6., 5., 2., 2., 6., 6., 6., 5., 3., 5., 3., 2., 6.,
         5., 3., 6., 6., 4., 6., 5., 2., 0., 5., 2., 0., 4., 3.],
        [8., 3., 4., 3., 2., 6., 6., 6., 3., 8., 6., 6., 5., 2., 2., 2., 4., 5.,
         6., 6., 5., 2., 4., 1., 2., 2., 2., 0., 0., 5., 8., 8.],
        [7., 2., 0., 5., 7., 2., 5., 3., 4., 6., 3., 8., 4., 6., 7., 2., 6., 6.,
         7., 2., 3., 8., 5., 3., 4., 5., 4., 1., 5., 3., 3., 8.],
        [0., 5., 4., 5., 5., 3., 6., 6., 6., 6., 2., 2., 2., 6., 4., 5., 8., 3.,
         5., 2., 2., 2., 0., 5., 5., 3., 5., 2., 2., 6., 0., 5.],
        [3., 8., 8., 3., 7., 2., 7., 2., 8., 3., 2., 2., 4., 6., 4., 5., 4., 1.,
         2., 2., 7., 2., 8., 8., 6., 6., 4., 3., 7., 2., 7., 2.],
        [8., 8., 2., 2., 2., 6., 5., 3., 8., 8., 0., 5., 2., 0., 4., 6., 2., 6.,
         4., 6., 2., 0., 5., 3., 5., 2., 7., 2., 3., 8., 8., 8.],
        [4., 5., 2., 6., 2., 2., 7., 2., 4., 6., 0., 5., 4., 6., 2., 0., 7., 2.,
         4., 1., 4., 1., 3., 8., 3., 

In [71]:
# binary 
def dec2bin(x, bits=None):
    """
    Convert decimal number to binary.
    :param x: decimal number
    :param bits: number of bits to use. If `None`, the minimum possible is used.
    :return: x in binary
    """
    if bits is None:
        bits = (x.max() + 1).log2().ceil().item()
    x = x.int()
    mask = 2 ** torch.arange(bits - 1, -1, -1).to(x.device, x.dtype)
    return x.unsqueeze(-1).bitwise_and(mask).ne(0).float()

x_binary = dec2bin(x)
x_binary = x_binary.permute(0, 2, 1)

print(x_binary[:10])

tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 1., 0.]],

        [[1., 1., 0.,  ..., 1., 1., 1.],
         [0., 0., 1.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 1., 1.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 1., 0.,  ..., 0., 0., 1.],
         [0., 0., 1.,  ..., 1., 1., 0.]],

        [[0., 0., 1.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 1., 0., 0.]],

        [[1., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 1., 0., 1.]]])


In [67]:
# Long output

x_long = x.long() + 1
print(x_long[:10])

tensor([[1, 3, 1, 3, 1, 1, 2, 3, 2, 2, 3, 1, 2, 3, 1, 3, 1, 3, 3, 1, 3, 1, 3, 1,
         2, 3, 2, 1, 2, 3, 2, 1, 1, 3, 3, 1, 2, 3, 2, 1, 3, 1, 3, 1, 2, 2, 3, 1,
         2, 3, 1, 3, 1, 1, 2, 3, 1, 3, 1, 1, 2, 2, 2, 1],
        [3, 3, 2, 1, 2, 2, 2, 1, 1, 3, 3, 1, 3, 1, 3, 1, 2, 1, 3, 3, 3, 1, 3, 1,
         2, 3, 1, 3, 1, 3, 1, 3, 2, 2, 2, 3, 3, 1, 3, 1, 2, 3, 1, 3, 2, 2, 1, 2,
         1, 3, 1, 3, 1, 3, 1, 1, 1, 1, 2, 3, 3, 3, 3, 3],
        [3, 2, 1, 3, 1, 1, 2, 3, 3, 2, 1, 3, 2, 3, 2, 1, 2, 2, 3, 1, 2, 1, 3, 3,
         2, 2, 3, 1, 3, 2, 1, 3, 3, 1, 3, 1, 3, 2, 1, 3, 2, 1, 3, 3, 2, 3, 2, 1,
         2, 2, 2, 3, 2, 2, 1, 2, 2, 3, 2, 1, 2, 1, 3, 3],
        [1, 1, 2, 3, 2, 2, 2, 3, 2, 3, 2, 1, 3, 1, 3, 1, 3, 1, 3, 1, 1, 3, 1, 3,
         1, 3, 3, 1, 2, 2, 2, 3, 3, 3, 2, 1, 2, 3, 1, 3, 1, 3, 1, 3, 1, 1, 2, 3,
         2, 3, 2, 1, 2, 3, 1, 3, 1, 3, 3, 1, 1, 1, 2, 3],
        [2, 1, 3, 3, 3, 3, 2, 1, 3, 2, 1, 3, 3, 2, 1, 3, 3, 3, 2, 1, 1, 3, 1, 3,
         2, 2, 3, 1, 2, 2, 2, 3, 2, 2, 

In [63]:
# decimal output
x_decimal = ((x[:, None] + 1) / num_features - 1) * 2
print(x_decimal[:10])

tensor([[[-1.3333,  0.0000, -1.3333,  ..., -0.6667, -0.6667, -1.3333]],

        [[ 0.0000,  0.0000, -0.6667,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000, -0.6667, -1.3333,  ..., -1.3333,  0.0000,  0.0000]],

        ...,

        [[-0.6667, -0.6667,  0.0000,  ..., -1.3333,  0.0000,  0.0000]],

        [[-1.3333,  0.0000, -1.3333,  ...,  0.0000, -0.6667, -1.3333]],

        [[-0.6667, -1.3333,  0.0000,  ..., -0.6667,  0.0000, -1.3333]]])


In [70]:
# onehot output
import torch.nn.functional as F

x_onehot = F.one_hot(
    x.long(),
    num_classes=num_features
).float()
x_onehot = x_onehot.permute(0, 2, 1)
print(x_onehot[:10])


tensor([[[1., 0., 1.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 1., 1., 1.]],

        [[0., 0., 1.,  ..., 1., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 1., 1.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 1., 1., 0.],
         [1., 1., 0.,  ..., 0., 0., 1.]],

        [[0., 1., 0.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 1., 0.]],

        [[0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 1.,  ..., 1., 0., 1.],
         [1., 1., 0.,  ..., 0., 0., 0.]]])


In [9]:
import pickle
from pprint import pprint

def loadall(filename):
    with open(filename, "rb") as f:
        while True:
            try:
                yield pickle.load(f)
            except EOFError:
                break

items = loadall("hierarchy-learning/test.pkl")

for item in items:
    pprint(item)

Namespace(device='cpu', dtype='float32', seed_init=0, seed_net=0, seed_trainset=0, dataset='hier1', ptr=51, pte=12, batch_size=128, scale_batch_size=0, background_noise=0, num_features=8, m=2, s=2, num_layers=2, num_classes=8, input_format='onehot', whitening=0, auto_regression=0, net='fcn', random_features=0, width=64, net_layers=3, filter_size=2, stride=2, batch_norm=0, bias=1, pmask=0.2, loss='cross_entropy', optim='sgd', scheduler='cosineannealing', lr=0.1, momentum=0.9, weight_decay=0.0005, reg_type='l2', epochs=250, zero_loss_epochs=0, zero_loss_threshold=0.01, rescale_epochs=0, alpha=1.0, stability=0, clustering_error=0, locality=0, save_init_net=1, save_best_net=1, save_last_net=1, save_dynamics=0, pickle='test.pkl', output='test.pkl')
{'args': Namespace(device='cpu', dtype='float32', seed_init=0, seed_net=0, seed_trainset=0, dataset='hier1', ptr=51, pte=12, batch_size=128, scale_batch_size=0, background_noise=0, num_features=8, m=2, s=2, num_layers=2, num_classes=8, input_form